# 06 Example - Continuous Scan

In [1]:
import pyscan as ps
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
import random

## Setup devices

In [2]:
devices = ps.ItemAttribute()

devices.v1 = ps.TestVoltage()


## Define a measure function

In [3]:
def measure_up_to_3D(expt):
    d = ps.ItemAttribute()

    d.x1 = expt.runinfo.scan0.i
    d.x2 = [expt.runinfo.scan0.i for _ in range(2)]
    d.x3 = [[expt.runinfo.scan0.i, expt.runinfo.scan0.i] for _ in range(2)]

    return d

## Experiments

### 1. 1D Scan with scan0 as ContinuousScan, using point data

In [4]:
runinfo = ps.RunInfo()
runinfo.measure_function = measure_up_to_3D
runinfo.scan0 = ps.ContinuousScan(n_max=100, dt=0.01)

expt = ps.Experiment(runinfo, devices)
expt.run()

Stopping Experiment
